### Problem:
Suppose we have next Game that happens on 3x5 screen/grid:

1. on the start 6 cells of the grid are occupied by symbols and all other are empty; 
2. on the start player has 3 spins, spins happen automatically;
3. for each spin, each empty cell of the screen has independent 0.05 probability to turn up as symbol;
4. for each spin during which at least one empty cell turned up as symbol, the current amount of spins that 
left is reset to the initial value of 3;
5. game is finished when either there are no spins left or the full screen is filled by symbols;
6. Check received theoretical results via simulation.

Find theoretical expected amount of symbols for final screen in that Game and its distribution.


In [40]:
import numpy as np
from tqdm import tqdm

# нас цікавить лише 15-6=9 клітинок сітки
def simulate():
    s = 3
    grid = np.zeros(9)
    while s>0 and grid.sum() < 9:
        for i in range(9):
            if np.random.rand()<=0.05 and grid[i]==0:
                grid[i] = 1
                s = 3
        s -= 1
    return grid.sum() 

history_amount_symbols = np.array([])
for _ in tqdm(range(1000000)):
    ss = simulate()
    history_amount_symbols = np.append(history_amount_symbols, ss)
print("expected amount of symbols for final screen = ", 6 + np.mean(history_amount_symbols))

100%|██████████| 1000000/1000000 [19:47<00:00, 841.82it/s] 

expected amount of symbols for final screen =  7.778268


Отже, правильна відповідь: 7.778

### Solution:
Переформулюємо нашу задачу: Маємо якусь випадкову величину X, яка набуває значень наприклад (6,6,7,8,8,...) \
Наша послідовність чисел має два випадки завершення:
1. (6,7,7,...,15) - тобто коли наша випадкова величина збільшилась до 15;
2. (6,7,8,...,9,10,10,10,10) - тобто коли отримали 4 однакових підряд числа ( на i-тому спіні отримали символ і наступні три нові спіни нічого не випало); 

З третього пункту умови можемо зробити висновок, що якщо символ випав, то він фіксується. Тобто вільних клітинок стає менше.
Очевидно, що на кожному кроці\спіні відбувається серія біноміальних випробувань по пустим клітинкам, що залишилися.
Тож розподіл буде задаватися наступним виразом: $ \Pr(X_{i+1} = X_{i} + n) = \binom {emptyleft}{n} 0.05^n * 0.95^{emptyleft-n}$

Питання: яке число буде останнім в середньому?

Для цього потрібно порахувати: ймовірність отримати 15 вкінці; ймовірність отримати підряд три 6, 7, 8, 9, 10, 11, 12, 13, 14; \
Наступним кроком за формулою мат сподівання для дискретних розподілів порахувати математичне сподівання ${\operatorname {E} (X)= 6 + \sum _{n}n\,p(n)},$\
Це і буде нашою відповіддю.


In [2]:
from math import comb
import numpy as np
from tqdm import tqdm

def p(n, empty_left):
    return comb(empty_left, n) * (0.05 ** n) * (0.95 ** (empty_left - n))

Пропоную перевірити наш підхід для спрощених задач: сітки з 6 зайнятими клітинками і 1,2,... вільними

Для початку перевіримо для сітки з 6 зайнятими і 1 вільною

In [59]:
def simulate():
    s = 3
    grid = np.zeros(1)
    while s>0 and grid.sum() < 1:
        for i in range(1):
            if np.random.rand()<=0.05 and grid[i]==0 :
                grid[i] = 1
                s = 3
        s -= 1
    return grid.sum() 

history_amount_symbols = np.array([])
for _ in tqdm(range(100000)):
    ss = simulate()
    history_amount_symbols = np.append(history_amount_symbols, ss)
print("expected amount of symbols for final screen with 1 empty, empirical E = ", 6 + np.mean(history_amount_symbols))


# (6,6,6) or (7) or (6,7) or (6,6,7) 
E = 6 +  0 * p(0, 1)**3 + 1 * ( p(0,1)**2 * p(1,1) + p(0,1) * p(1,1) + p(1,1) )
print(f'theory {E=}')

  0%|          | 0/100000 [00:00<?, ?it/s]

100%|██████████| 100000/100000 [00:03<00:00, 25065.73it/s]

expected amount of symbols for final screen with 1 empty, empirical E =  6.1411
theory E=6.142625


Результати однакові, отже можемо припустити, що працює. Для впевненості перевіримо тепер для 6 зайнятих і 2 вільних.

In [22]:
def simulate():
    s = 3
    grid = np.zeros(2)
    while s>0 and grid.sum() < 2:
        for i in range(2):
            if np.random.rand()<=0.05 and grid[i]==0 :
                grid[i] = 1
                s = 3
        s -= 1
    return grid.sum() 

history_amount_symbols = np.array([])
for _ in tqdm(range(100000)):
    ss = simulate()
    history_amount_symbols = np.append(history_amount_symbols, ss)
print("expected amount of symbols for final screen with 2 empty, empirical E = ", 6 + np.mean(history_amount_symbols))


# (6,6,6)
p_0 = p(0, 2)**3 
# (7,7,7,7) or (6,7,7,7,7) or (6,6,7,7,7,7)
p_1 = p(1,2) * p(0,1)**3 + p(0,2) * p(1,2) * p(0,1)**3 + p(0,2)**2 * p(1,2) * p(0,1)**3
# (8) or (6,8) or (7,8) or (6,6,8) or (6,7,8) or (7,7,8) or (6,7,7,8) or (6,7,7,7,8) or (7,7,7,8) or (6,6,7,7,8) or (6,6,7,7,7,8)
p_2 = p(2,2) + p(0,2) * p(2,2) + p(1,2) * p(1,1) + p(0,2)**2 * p(2,2) + p(0,2) * p(1,2) * p(1,1) + p(1,2) * p(0,1) * p(1,1) + p(0,2) * p(1,2) * p(0,1) * p(1,1) + p(0,2) * p(1,2) * p(0,1) * p(0,1) * p(1,1) + p(1,2) * p(0,1)**2 * p(1,1) + p(0,2)**2 * p(1,2) * p(0,1) * p(1,1) + p(0,2)**2 * p(1,2) * p(0,1)**2 * p(1,1)

E = 6 +  0 * p_0 + 1 * p_1 + 2 * p_2
print(f'theory {E=}')

  0%|          | 0/100000 [00:00<?, ?it/s]

100%|██████████| 100000/100000 [00:04<00:00, 22421.95it/s]

expected amount of symbols for final screen with 2 empty, empirical E =  6.2998899999999995
theory E=6.300776552183594
